<img src="../rsag_convex.png" alt="algoconvex" />
<img src="../x_update.png" alt="x_update" />
<img src="../mean.png" alt="mean" />
<img src="../rsag_composite.png" alt="algo" />

__Parameters :__
- $\alpha$: (1-$\alpha$) weight of aggregated x on current state, i.e. momentum
- $\lambda$: learning rate
- $\beta$: change for aggregated x
- $p_k$ termination probability



In [1]:
from torch.optim.optimizer import Optimizer, required
import torch
import copy
from torch.nn import functional as F
from torch import nn
import numpy as np

In [2]:
import torch.utils.data as data_utils

In [3]:
print('Using PyTorch version:', torch.__version__)
if torch.cuda.is_available():
    print('Using GPU, device name:', torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print('No GPU found, using CPU instead.') 
    device = torch.device('cpu')

Using PyTorch version: 2.1.2+cu121
Using GPU, device name: NVIDIA GeForce GTX 1660 Ti


In [1]:
import path
import sys
sys.path.append('../')
from models import MLP
from optimizers import RSAG, AccSGD
from util import get_mnist, calc_accuracy, train_model


ModuleNotFoundError: No module named 'get_mnist'

## Run

In [10]:
loaders = get_mnist()


In [ ]:
loss_function = torch.nn.CrossEntropyLoss()
model = MLP().to(device)
print(model)

In [28]:

optimizer = RSAG(model.parameters(), lr=1e-4, alpha=1e-2, beta=.1)
# optimizer = torch.optim.Adagrad(mlp.parameters(), lr=1e-4)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [96]:
model = MLP().to(device)
print(model)

loss_function = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, nesterov=True, momentum=0.9)
optimizer = RSAG(model.parameters(), lr=1e-4, alpha=.9, beta=9e-5)

train_model(model, loss_function, optimizer, loaders, device, n_epochs=5)

MLP(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Starting Epoch 1
Epoch 1 finished
loss 2.2992
Accuracy:  10.4313
Validation loss 2.2984
Validation Accuracy:  10.4167
Starting Epoch 2
Epoch 2 finished
loss 2.2972
Accuracy:  11.4125
Validation loss 2.2965
Validation Accuracy:  11.2917
Starting Epoch 3
Epoch 3 finished
loss 2.2953
Accuracy:  12.4833
Validation loss 2.2945
Validation Accuracy:  12.2917
Starting Epoch 4
Epoch 4 finished
loss 2.2933
Accuracy:  13.5208
Validation loss 2.2925
Validation Accuracy:  13.1917
Starting Epoch 5
Epoch 5 finished
loss 2.2913
Accuracy:  14.6479
Validation loss 2.2906
Validation Accuracy:  14.2917
Training has completed


{'loss': [2.2991838619112968,
  2.297221561272939,
  2.295258607963721,
  2.2932929024100304,
  2.2913220246632893],
 'accuracy': [10.43125,
  11.4125,
  12.483333333333333,
  13.520833333333334,
  14.647916666666667],
 'v_loss': [2.298423532644908,
  2.296466801563899,
  2.294509494304657,
  2.292549298206965,
  2.2905831933021545],
 'v_accuracy': [10.416666666666666,
  11.291666666666666,
  12.291666666666666,
  13.191666666666666,
  14.291666666666666],
 'v_loss_std': [0.003215891868610463,
  0.003290792243270636,
  0.0034047087296312406,
  0.0031955316909583627,
  0.003157745742080987],
 'v_accuracy_std': [2.976528552226943,
  2.9080229060690392,
  3.2516555612316766,
  3.5643976427379083,
  3.6954833543424632],
 'loss_std': [0.003592137767668389,
  0.0036445462901459596,
  0.0036359242979697877,
  0.003290839702223381,
  0.0032090308513339333],
 'accuracy_std': [2.9733998672955737,
  3.4084224723469947,
  3.4326455621802197,
  3.386182605662148,
  3.60078056256171]}

In [81]:
def train_with_hyperparameters(alpha_values, lr_values, save_log=False):
    loss_function = torch.nn.CrossEntropyLoss()
    best_alpha, best_lr = 0.0, 0.0
    best_accuracy = 0.0
    v_accs, acc_std, v_loss, loss_std = [], [], [], []
    acc, loss = [], []
    
    for alpha in alpha_values:
        for lr in lr_values:
            beta = lr * alpha
            
            print(f"----------- Training with alpha={alpha}, lr={lr} -----------------")
            
            model = MLP().to(device)
            optimizer = RSAG(model.parameters(), lr=lr, alpha=alpha, beta=beta)
            log = train_model(model, loaders, optimizer, loss_function, device, epochs=20)
            
            if log['v_accuracy'][-1] > best_accuracy:
                print(f"Found a new best accuracy: {log['v_accuracy'][-1]}")
                print(f"best alpha: {alpha}, best lr: {lr}")
                best_accuracy = log['v_accuracy'][-1]
                best_alpha = alpha
                best_lr = lr
            
            v_accs.append(log['v_accuracy'])
            acc_std.append(log['v_accuracy_std'])
            v_loss.append(log['v_loss'])
            loss_std.append(log['v_loss_std'])
            acc.append(log['accuracy'])
            loss.append(log['loss'])
            

    
    return best_alpha, best_lr, v_accs, acc_std, v_loss, loss_std, acc, loss


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 58)

In [46]:

optimizer = RSAG(model.parameters(), lr=1e-4, alpha=.9, beta=9e-5)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, nesterov=True, momentum=0.9)
train_model(model, loaders, optimizer, loss_function, device, epochs=5)

Starting Epoch 1
Epoch 1 finished
loss 4.6072
Accuracy:  7.4900
Starting Epoch 2
Epoch 2 finished
loss 4.6027
Accuracy:  8.8633
Starting Epoch 3
Epoch 3 finished
loss 4.5982
Accuracy:  10.3017
Starting Epoch 4
Epoch 4 finished
loss 4.5937
Accuracy:  11.6917
Starting Epoch 5
Epoch 5 finished
loss 4.5893
Accuracy:  13.1383
Training has completed
